In [1]:
# import libraries
import pandas as pd
import gffpandas.gffpandas as gffpd

In [2]:
# Upload assembly gff file
asm = gffpd.read_gff3("adj_primary.fasta_liftoff_out.gff3")
# Retain only gene features in assembly gff file
gene_asm = asm.filter_feature_of_type(['gene']).to_tsv('primary_temp.tsv')
gene_asm = pd.read_table('primary_temp.tsv')
# Split attributes column into two columns
gene_asm[['gene_id', 'attributes']] = gene_asm['attributes'].str.split(pat=';', expand=True, n=1)
# Filter columns to retain only gene_id, seq_id, type, start, end, strand
gene_asm_filt = gene_asm.filter(['seq_id', 'type', 'start', 'end', 'strand', 'gene_id'])
# add 'asm' prefix to column names
gene_asm_filt = gene_asm_filt.add_prefix('asm_')

In [3]:
print(gene_asm_filt)

      asm_seq_id asm_type  asm_start   asm_end asm_strand  \
0         chr_18     gene      50272     54591          +   
1         chr_18     gene      55313     56493          +   
2         chr_18     gene      75253     76046          +   
3         chr_18     gene      79894     86835          +   
4         chr_18     gene      87616    111056          +   
...          ...      ...        ...       ...        ...   
26965     chr_12     gene   26344665  26375113          +   
26966     chr_12     gene   26353254  26353372          +   
26967     chr_12     gene   26375023  26388046          -   
26968     chr_12     gene   26400552  26433702          +   
26969     chr_12     gene   26435449  26451406          -   

                asm_gene_id  
0      ID=gene-LOC100244872  
1      ID=gene-LOC100248269  
2      ID=gene-LOC132254911  
3      ID=gene-LOC100267182  
4      ID=gene-LOC100244867  
...                     ...  
26965  ID=gene-LOC100245514  
26966  ID=gene-LOC132254831

In [4]:
# upload reference gff file
ref = gffpd.read_gff3("adj_reference.gff")
# retain only gene features in reference gff file
gene_ref = ref.filter_feature_of_type(['gene']).to_tsv('reference_temp.tsv')
# split attributes column into two columns
gene_ref = pd.read_table('reference_temp.tsv')
gene_ref[['gene_id', 'attributes']] = gene_ref['attributes'].str.split(pat=';', expand=True, n=1)
# filter columns to retain only gene_id, seq_id, type, start, end, strand, attributes
gene_ref_filt = gene_ref.filter(['seq_id', 'type', 'start', 'end', 'strand', 'gene_id', 'attributes'])
# add 'ref' prefix to column names
gene_ref_filt = gene_ref_filt.add_prefix('ref_')

# Define the regular expression to match 'chr_**' where '**' is a number between '01' and '19'
pattern = r'^chr_([0][1-9]|1[0-9])$'

# Filter the DataFrame
gene_ref_filt = gene_ref_filt[gene_ref_filt['ref_seq_id'].str.contains(pattern)]

# Print the filtered DataFrame
print(gene_ref_filt)


      ref_seq_id ref_type  ref_start   ref_end ref_strand  \
0         chr_01     gene      16041     32947          +   
1         chr_01     gene      37289     41411          +   
2         chr_01     gene      53915     63050          +   
3         chr_01     gene      66246     66840          -   
4         chr_01     gene      88278    104293          -   
...          ...      ...        ...       ...        ...   
27899     chr_19     gene   27035382  27035763          +   
27900     chr_19     gene   27043031  27063568          +   
27901     chr_19     gene   27064010  27078905          +   
27902     chr_19     gene   27111234  27121227          +   
27903     chr_19     gene   27143977  27145739          -   

                ref_gene_id                                     ref_attributes  
0      ID=gene-LOC104879287  Dbxref=GeneID:104879287;Name=LOC104879287;desc...  
1      ID=gene-LOC100259472  Dbxref=GeneID:100259472;Name=LOC100259472;desc...  
2      ID=gene-LOC100257

/tmp/ipykernel_27100/3273317542.py:17: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  gene_ref_filt = gene_ref_filt[gene_ref_filt['ref_seq_id'].str.contains(pattern)]


In [5]:
# merge filtered reference and assembly gff files and sort by ref_seq_id and ref_start
merged = pd.merge(gene_asm_filt, gene_ref_filt, how="inner", left_on='asm_gene_id', right_on='ref_gene_id')

In [6]:
print(merged)

      asm_seq_id asm_type  asm_start   asm_end asm_strand  \
0         chr_18     gene      50272     54591          +   
1         chr_18     gene      55313     56493          +   
2         chr_18     gene      75253     76046          +   
3         chr_18     gene      79894     86835          +   
4         chr_18     gene      87616    111056          +   
...          ...      ...        ...       ...        ...   
26846     chr_12     gene   26344665  26375113          +   
26847     chr_12     gene   26353254  26353372          +   
26848     chr_12     gene   26375023  26388046          -   
26849     chr_12     gene   26400552  26433702          +   
26850     chr_12     gene   26435449  26451406          -   

                asm_gene_id ref_seq_id ref_type  ref_start   ref_end  \
0      ID=gene-LOC100244872     chr_18     gene   36624063  36628394   
1      ID=gene-LOC100248269     chr_18     gene   36622149  36623341   
2      ID=gene-LOC132254911     chr_13     gene   2

In [7]:
# Group by 'asm_seq_id' and count the number of rows in each group
row_counts = merged.groupby('asm_seq_id').size()

# Print the result
print(row_counts)

asm_seq_id
chr_01    1618
chr_02    1123
chr_03    1174
chr_04    1524
chr_05    1506
chr_06    1437
chr_07    1965
chr_08    1614
chr_09    1143
chr_10    1356
chr_11    1138
chr_12    1231
chr_13    1514
chr_14    1703
chr_15     972
chr_16    1175
chr_17    1329
chr_18    2109
chr_19    1220
dtype: int64


In [8]:
# Group by 'asm_seq_id' and count the number of rows in each group
row_counts_ref = gene_ref_filt.groupby('ref_seq_id').size()

# Print the result
print(row_counts_ref)

ref_seq_id
chr_01    1620
chr_02    1166
chr_03    1192
chr_04    1542
chr_05    1554
chr_06    1477
chr_07    2025
chr_08    1640
chr_09    1208
chr_10    1476
chr_11    1123
chr_12    1335
chr_13    1633
chr_14    1783
chr_15    1071
chr_16    1294
chr_17    1350
chr_18    2173
chr_19    1242
dtype: int64


In [9]:
# Filter the rows where asm_seq_id is different from ref_seq_id
diff_seq_id_df = merged[merged['asm_seq_id'] != merged['ref_seq_id']]

# Print the filtered DataFrame
print(diff_seq_id_df)


      asm_seq_id asm_type  asm_start   asm_end asm_strand  \
2         chr_18     gene      75253     76046          +   
44        chr_18     gene     521246    523503          +   
54        chr_18     gene     755682    756076          -   
55        chr_18     gene     756777    761140          -   
58        chr_18     gene     827910    831535          +   
...          ...      ...        ...       ...        ...   
26814     chr_12     gene   24830664  24831344          +   
26815     chr_12     gene   24898074  24899178          +   
26818     chr_12     gene   25328122  25328597          -   
26820     chr_12     gene   25679329  25680850          +   
26823     chr_12     gene   25803768  25803820          +   

                asm_gene_id ref_seq_id ref_type  ref_start   ref_end  \
2      ID=gene-LOC132254911     chr_13     gene   20719485  20720479   
44     ID=gene-LOC104879996     chr_08     gene    4612852   4615109   
54     ID=gene-LOC132254321     chr_09     gene   1

In [10]:
# Save the filtered DataFrame to a new variable
filtered_df = diff_seq_id_df

# Optionally, save the filtered DataFrame to a CSV file
filtered_df.to_csv('traslocations_primary.csv', index=False)

In [11]:
# Upload assembly gff file
asm = gffpd.read_gff3("adj_hap1.fasta_liftoff_out.gff3")
# Retain only gene features in assembly gff file
gene_asm = asm.filter_feature_of_type(['gene']).to_tsv('hap1_temp.tsv')
gene_asm = pd.read_table('hap1_temp.tsv')
# Split attributes column into two columns
gene_asm[['gene_id', 'attributes']] = gene_asm['attributes'].str.split(pat=';', expand=True, n=1)
# Filter columns to retain only gene_id, seq_id, type, start, end, strand
gene_asm_filt = gene_asm.filter(['seq_id', 'type', 'start', 'end', 'strand', 'gene_id'])
# add 'asm' prefix to column names
gene_asm_filt = gene_asm_filt.add_prefix('asm_')

In [12]:
print(gene_asm_filt)

              asm_seq_id asm_type  asm_start   asm_end asm_strand  \
0      HAP1_scaffold_102     gene       5976     14947          -   
1      HAP1_scaffold_104     gene      15787     18487          +   
2      HAP1_scaffold_106     gene      21770     24500          +   
3      HAP1_scaffold_106     gene      25614     26952          -   
4      HAP1_scaffold_106     gene      54068     55090          -   
...                  ...      ...        ...       ...        ...   
26951             chr_04     gene   26176444  26183701          -   
26952             chr_04     gene   26196737  26197791          -   
26953             chr_04     gene   26203407  26207802          -   
26954             chr_04     gene   26233543  26234383          +   
26955             chr_04     gene   26240154  26241656          -   

                asm_gene_id  
0      ID=gene-LOC100256473  
1      ID=gene-LOC109124167  
2      ID=gene-LOC100242403  
3      ID=gene-LOC100264688  
4      ID=gene-LOC132

In [13]:
print(gene_ref_filt)

      ref_seq_id ref_type  ref_start   ref_end ref_strand  \
0         chr_01     gene      16041     32947          +   
1         chr_01     gene      37289     41411          +   
2         chr_01     gene      53915     63050          +   
3         chr_01     gene      66246     66840          -   
4         chr_01     gene      88278    104293          -   
...          ...      ...        ...       ...        ...   
27899     chr_19     gene   27035382  27035763          +   
27900     chr_19     gene   27043031  27063568          +   
27901     chr_19     gene   27064010  27078905          +   
27902     chr_19     gene   27111234  27121227          +   
27903     chr_19     gene   27143977  27145739          -   

                ref_gene_id                                     ref_attributes  
0      ID=gene-LOC104879287  Dbxref=GeneID:104879287;Name=LOC104879287;desc...  
1      ID=gene-LOC100259472  Dbxref=GeneID:100259472;Name=LOC100259472;desc...  
2      ID=gene-LOC100257

In [14]:
# merge filtered reference and assembly gff files and sort by ref_seq_id and ref_start
merged = pd.merge(gene_asm_filt, gene_ref_filt, how="inner", left_on='asm_gene_id', right_on='ref_gene_id')

In [15]:
print(merged)

              asm_seq_id asm_type  asm_start   asm_end asm_strand  \
0      HAP1_scaffold_102     gene       5976     14947          -   
1      HAP1_scaffold_104     gene      15787     18487          +   
2      HAP1_scaffold_106     gene      21770     24500          +   
3      HAP1_scaffold_106     gene      25614     26952          -   
4      HAP1_scaffold_106     gene      54068     55090          -   
...                  ...      ...        ...       ...        ...   
26847             chr_04     gene   26176444  26183701          -   
26848             chr_04     gene   26196737  26197791          -   
26849             chr_04     gene   26203407  26207802          -   
26850             chr_04     gene   26233543  26234383          +   
26851             chr_04     gene   26240154  26241656          -   

                asm_gene_id ref_seq_id ref_type  ref_start   ref_end  \
0      ID=gene-LOC100256473     chr_04     gene   10242714  10251672   
1      ID=gene-LOC109124167

In [16]:
# Group by 'asm_seq_id' and count the number of rows in each group
row_counts = merged.groupby('asm_seq_id').size()

# Print the result
print(row_counts)

asm_seq_id
HAP1_scaffold_102       1
HAP1_scaffold_104       1
HAP1_scaffold_106       3
HAP1_scaffold_108       1
HAP1_scaffold_125       7
                     ... 
chr_15                976
chr_16               1137
chr_17               1266
chr_18               2106
chr_19               1212
Length: 133, dtype: int64


In [17]:
#save row count to tsv
row_counts.to_csv('row_counts_hap1.tsv', sep='\t')

In [18]:
# Filter the rows where asm_seq_id is different from ref_seq_id
diff_seq_id_df = merged[merged['asm_seq_id'] != merged['ref_seq_id']]

# Print the filtered DataFrame
print(diff_seq_id_df)

              asm_seq_id asm_type  asm_start   asm_end asm_strand  \
0      HAP1_scaffold_102     gene       5976     14947          -   
1      HAP1_scaffold_104     gene      15787     18487          +   
2      HAP1_scaffold_106     gene      21770     24500          +   
3      HAP1_scaffold_106     gene      25614     26952          -   
4      HAP1_scaffold_106     gene      54068     55090          -   
...                  ...      ...        ...       ...        ...   
26634             chr_04     gene   24162079  24165074          +   
26646             chr_04     gene   24260382  24260903          +   
26647             chr_04     gene   24263936  24264307          -   
26718             chr_04     gene   24940049  24942006          +   
26719             chr_04     gene   24942183  24948696          +   

                asm_gene_id ref_seq_id ref_type  ref_start   ref_end  \
0      ID=gene-LOC100256473     chr_04     gene   10242714  10251672   
1      ID=gene-LOC109124167

In [19]:
# Save the filtered DataFrame to a new variable
filtered_df = diff_seq_id_df

# Optionally, save the filtered DataFrame to a CSV file
filtered_df.to_csv('traslocations_hap1.csv', index=False)

In [29]:
# Upload assembly gff file
asm = gffpd.read_gff3("adj_hap2.fasta_liftoff_out.gff3")
# Retain only gene features in assembly gff file
gene_asm = asm.filter_feature_of_type(['gene']).to_tsv('hap2_temp.tsv')
gene_asm = pd.read_table('hap2_temp.tsv')
# Split attributes column into two columns
gene_asm[['gene_id', 'attributes']] = gene_asm['attributes'].str.split(pat=';', expand=True, n=1)
# Filter columns to retain only gene_id, seq_id, type, start, end, strand
gene_asm_filt = gene_asm.filter(['seq_id', 'type', 'start', 'end', 'strand', 'gene_id'])
# add 'asm' prefix to column names
gene_asm_filt = gene_asm_filt.add_prefix('asm_')

In [30]:
print(gene_asm_filt)

              asm_seq_id asm_type  asm_start   asm_end asm_strand  \
0      HAP2_scaffold_100     gene      48244     49143          -   
1      HAP2_scaffold_100     gene      49624     50991          -   
2      HAP2_scaffold_103     gene      20992     22203          +   
3      HAP2_scaffold_103     gene      64641     66090          -   
4      HAP2_scaffold_105     gene      20149     23402          -   
...                  ...      ...        ...       ...        ...   
27031             chr_10     gene   25289839  25293214          -   
27032             chr_10     gene   25295813  25299698          +   
27033             chr_10     gene   25304475  25307598          +   
27034             chr_10     gene   25308893  25311988          +   
27035             chr_10     gene   25319606  25321991          +   

                asm_gene_id  
0      ID=gene-LOC109123996  
1      ID=gene-LOC104877469  
2      ID=gene-LOC104878040  
3      ID=gene-LOC100265481  
4      ID=gene-LOC100

In [31]:
print(gene_ref_filt)

      ref_seq_id ref_type  ref_start   ref_end ref_strand  \
0         chr_01     gene      16041     32947          +   
1         chr_01     gene      37289     41411          +   
2         chr_01     gene      53915     63050          +   
3         chr_01     gene      66246     66840          -   
4         chr_01     gene      88278    104293          -   
...          ...      ...        ...       ...        ...   
27899     chr_19     gene   27035382  27035763          +   
27900     chr_19     gene   27043031  27063568          +   
27901     chr_19     gene   27064010  27078905          +   
27902     chr_19     gene   27111234  27121227          +   
27903     chr_19     gene   27143977  27145739          -   

                ref_gene_id                                     ref_attributes  
0      ID=gene-LOC104879287  Dbxref=GeneID:104879287;Name=LOC104879287;desc...  
1      ID=gene-LOC100259472  Dbxref=GeneID:100259472;Name=LOC100259472;desc...  
2      ID=gene-LOC100257

In [33]:
# merge filtered reference and assembly gff files and sort by ref_seq_id and ref_start
merged = pd.merge(gene_asm_filt, gene_ref_filt, how="inner", left_on='asm_gene_id', right_on='ref_gene_id')

In [34]:
print(merged)

              asm_seq_id asm_type  asm_start   asm_end asm_strand  \
0      HAP2_scaffold_100     gene      48244     49143          -   
1      HAP2_scaffold_100     gene      49624     50991          -   
2      HAP2_scaffold_103     gene      20992     22203          +   
3      HAP2_scaffold_103     gene      64641     66090          -   
4      HAP2_scaffold_105     gene      20149     23402          -   
...                  ...      ...        ...       ...        ...   
26908             chr_10     gene   25289839  25293214          -   
26909             chr_10     gene   25295813  25299698          +   
26910             chr_10     gene   25304475  25307598          +   
26911             chr_10     gene   25308893  25311988          +   
26912             chr_10     gene   25319606  25321991          +   

                asm_gene_id ref_seq_id ref_type  ref_start   ref_end  \
0      ID=gene-LOC109123996     chr_15     gene   18598753  18607538   
1      ID=gene-LOC104877469

In [35]:
# Group by 'asm_seq_id' and count the number of rows in each group
row_counts = merged.groupby('asm_seq_id').size()

# Print the result
print(row_counts)

asm_seq_id
HAP2_scaffold_100       2
HAP2_scaffold_103       2
HAP2_scaffold_105       4
HAP2_scaffold_110       4
HAP2_scaffold_115       1
                     ... 
chr_15                952
chr_16               1169
chr_17               1244
chr_18               2077
chr_19               1163
Length: 151, dtype: int64


In [36]:
#save row count to tsv
row_counts.to_csv('row_counts_hap2.tsv', sep='\t')

In [37]:
# Filter the rows where asm_seq_id is different from ref_seq_id
diff_seq_id_df = merged[merged['asm_seq_id'] != merged['ref_seq_id']]

# Print the filtered DataFrame
print(diff_seq_id_df)

              asm_seq_id asm_type  asm_start   asm_end asm_strand  \
0      HAP2_scaffold_100     gene      48244     49143          -   
1      HAP2_scaffold_100     gene      49624     50991          -   
2      HAP2_scaffold_103     gene      20992     22203          +   
3      HAP2_scaffold_103     gene      64641     66090          -   
4      HAP2_scaffold_105     gene      20149     23402          -   
...                  ...      ...        ...       ...        ...   
26705             chr_10     gene   23399415  23400161          +   
26757             chr_10     gene   23901149  23901782          -   
26759             chr_10     gene   23941366  23941777          -   
26828             chr_10     gene   24615059  24617033          +   
26829             chr_10     gene   24617131  24618539          +   

                asm_gene_id ref_seq_id ref_type  ref_start   ref_end  \
0      ID=gene-LOC109123996     chr_15     gene   18598753  18607538   
1      ID=gene-LOC104877469

In [38]:
# Save the filtered DataFrame to a new variable
filtered_df = diff_seq_id_df

# Optionally, save the filtered DataFrame to a CSV file
filtered_df.to_csv('traslocations_hap2.csv', index=False)